In [33]:
import pandas as pd
import re
from datetime import date
import pickle
from bs4 import BeautifulSoup
import gensim
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [34]:
df = pd.read_excel('social_content/tw/mms_uk_ugc_tw.xls',skiprows=1)
filename = 'mms_uk_ugc_tw'

In [35]:
df = df.rename(columns = {'Contents':'text'})

In [36]:
text = []
i = 0
for item in df.text:
    item = re.sub(r'http\S+','', df.text[i], flags=re.MULTILINE)
    text.append(item)
    i= i+1

In [37]:
df['text'] = text

In [38]:
# Text Parsing

## stem and tokenize
texts = gensim.parsing.preprocess_documents(df.text.values)

# prase detection
bigram_model = gensim.models.phrases.Phrases(texts)
bigrams = [bigram_model[t] for t in texts]

# prepare corpus and dictionary for LDA
dictionary = gensim.corpora.Dictionary(bigrams)
corpus = [dictionary.doc2bow(b) for b in bigrams]

In [43]:
# LDA Model
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=8, passes=20)
lda.save(filename + date.today().isoformat()+'.model')

In [44]:
# Visualize
prepared_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
prepared_data

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.038248 -0.090290       1        1  16.425369
0      0.053133 -0.009050       2        1  13.414720
7     -0.081059  0.113029       3        1  13.348796
4     -0.144405 -0.024343       4        1  13.276008
3     -0.009208 -0.015774       5        1  12.259459
6      0.097361  0.116179       6        1  11.632627
1      0.035067 -0.037204       7        1   9.978980
5      0.010862 -0.052547       8        1   9.664046, topic_info=     Category       Freq                             Term       Total  \
term                                                                    
3     Default  82.000000                            world   82.000000   
208   Default  43.000000                           london   43.000000   
12    Default  41.000000                             like   41.000000   
18    Default  19.000000                              mar   19.000000   
325   Default   9.000000                        hot_cross    9.000000   
23    Default  13.000000                        crispi_m’   13.000000   
177   Default  20.000000                      like_youtub   20.000000   
324   Default   7.000000                      bun_flavour    7.000000   
47    Default  16.000000                              egg   16.000000   
565   Default   7.000000                  colour_christma    7.000000   
568   Default   7.000000                         m’_creat    7.000000   
564   Default   7.000000  amsterdamschiphol_londongatwick    7.000000   
563   Default   7.000000                  airport_shopper    7.000000   
569   Default   7.000000                      special_tin    7.000000   
567   Default   7.000000                   mar_osloaiport    7.000000   
566   Default   7.000000            londonheathrow_mmstin    7.000000   
533   Default  20.000000                            video   20.000000   
111   Default  28.000000                              bag   28.000000   
1537  Default   8.000000                              usb    8.000000   
157   Default  23.000000                            thank   23.000000   
4     Default  12.000000                           eminem   12.000000   
160   Default  11.000000                           fridai   11.000000   
16    Default  14.000000                           easter   14.000000   
55    Default  34.000000                              eat   34.000000   
414   Default  10.000000                            drive   10.000000   
17    Default   7.000000                       egg_easter    7.000000   
970   Default  13.000000                             week   13.000000   
54    Default  64.000000                           chocol   64.000000   
239   Default  17.000000                           crispi   17.000000   
135   Default  15.000000                             look   15.000000   
...       ...        ...                              ...         ...   
301    Topic8   2.102739                           smarti    4.236769   
326    Topic8   3.953155                           launch    8.493924   
252    Topic8   2.792594                  youtub_playlist    6.363486   
53     Topic8   4.019557                            brown   10.739330   
245    Topic8   2.792526                         ad_video    6.363487   
534    Topic8   3.473706                             awai    9.135216   
970    Topic8   4.149042                             week   13.015429   
195    Topic8   3.473189                           famili    9.892081   
228    Topic8   5.847105                             time   26.067118   
188    Topic8   6.880059                              new   44.870892   
413    Topic8   6.139874                              dai   39.092552   
808    Topic8   2.795024                        breakfast    7.852945   
742    Topic8   2.795471                             show    7.871255   
437    Topic8   2.795387                          present  

In [50]:
list(df[df.text.str.lower().str.contains('radcliff') ==True].text)

['SEEING DANIEL RADCLIFFE, M&M WORLD & THE FUNNIEST DAY IN New York! VLOGM...  via @YouTube',
 'I liked a @YouTube video  SEEING DANIEL RADCLIFFE, M&M WORLD & THE FUNNIEST DAY IN New York! VLOGMAS DAY 10!',
 'I liked a @YouTube video  SEEING DANIEL RADCLIFFE, M&M WORLD & THE FUNNIEST DAY IN New York! VLOGMAS DAY 10!',
 'I liked a @YouTube video  SEEING DANIEL RADCLIFFE, M&M WORLD & THE FUNNIEST DAY IN New York! VLOGMAS DAY 10!']